In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import requests
import pandas as pd
import csv
import re
import string
import nltk

from bs4 import BeautifulSoup
from numpy import NAN
from nltk.stem import WordNetLemmatizer

In [ ]:
# Define a user agent to qualify request
user_agent = ({"User-Agent":
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                    AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/90.0.4430.212 Safari/537.36",
               "Accept-Language": "en-US, en;q=0.5"})

In [ ]:
# Read in the csv file with wikipedia links for each observation
wk = pd.read_csv("/content/drive/MyDrive/Priceline/JintongYu/wiki_links.csv", usecols=['city_name', 'wiki_link'])

In [ ]:
wk.head()

,city_name,wiki_link
0,ROME,https://en.wikipedia.org/wiki/Rome
1,BARCELONA,https://en.wikipedia.org/wiki/Barcelona
2,ISTANBUL,https://en.wikipedia.org/wiki/Istanbul
3,MILAN,https://en.wikipedia.org/wiki/Milan
4,FLORENCE,https://en.wikipedia.org/wiki/Florence


In [ ]:
# Convert the data frame to a list
wk_ls = wk["wiki_link"].tolist()

In [ ]:
# Define a function to extract raw text data from a Wikipedia page
def scrape_wiki(link):
  try:
    # Send an HTTP GET request to the link and retrieve the HTML content
    response = requests.get(link, headers=user_agent)
    html_content = response.content
    
    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract text data
    text = ''
    for paragraph in soup.find_all('p'):
      text += paragraph.text
    
    # Create a dictionary for storing text
    return{
        "paragraph": text
    }
  except:
    return{
        "paragraph": NAN
    }

In [ ]:
# Loop through all Wikipedia links for extracting text data
results = []
for link in wk_ls:
  results.append(scrape_wiki(link))

In [ ]:
# Convert the dictionary to a data frame
wk_df = pd.DataFrame.from_dict(results)

In [ ]:
# Save the scraped raw text to a CSV file
wk_df.to_csv("/content/drive/MyDrive/Priceline/JintongYu/wiki_raw.csv")

In [ ]:
# Clean the raw text
# For removing stopwords in text
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Define a function for text cleaning that can be applied for a column in a data frame
def clean_text(df, column):
    """
    Preprocesses a text column in a pandas DataFrame.
    """
    # Convert to lowercase
    df[column] = df[column].str.lower()
    
    # Remove punctuation
    df[column] = df[column].str.replace('[{}]'.format(string.punctuation), '')
    
    # Remove numbers
    df[column] = df[column].str.replace('\d+', '')

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    df[column] = df[column].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

    # Remove stop words
    nltk.download('stopwords')
    stop_words = set(nltk.corpus.stopwords.words('english'))
    df[column] = df[column].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    # Remove selected words --- this step is supposed to be modified for different use cases
    df[column] = df[column].str.replace(' wa ', ' ')
    df[column] = df[column].str.replace(' ha ', ' ')
    df[column] = df[column].str.replace(' th ', ' ')
    df[column] = df[column].str.replace(' km ', ' ')
    df[column] = df[column].str.replace(' mile ', ' ')
    df[column] = df[column].str.replace(' mi ', ' ')
    df[column] = df[column].str.replace(' st ', ' ')
    df[column] = df[column].str.replace('°c', ' ')
    df[column] = df[column].str.replace('°f', ' ')
    df[column] = df[column].str.replace(' also ', ' ')
    #df[column] = df[column].str.replace('[^\x01-\x7F]', '') # Remove non-English characters

    # Strip whitespace
    df[column] = df[column].str.strip()

    return df

In [ ]:
# Apply the clean_text function to the raw text
clean_text(wk_df, "paragraph")

<ipython-input-51-6660f075362a>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('[{}]'.format(string.punctuation), '')
<ipython-input-51-6660f075362a>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('\d+', '')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,paragraph
0,caput mundi latinthe capital world rome italia...
1,barcelona ˌbɑːrsəˈloʊnə listen barsəlohnə cata...
2,istanbul ˌɪstænˈbʊl istanbuul u ˈɪstænbʊl ista...
3,milan mɪˈlæn milan u mɪˈlɑːn milahn lombard mi...
4,florence ˈflɒrəns florrənss italian firenze fi...
...,...
175,seoul soʊl korean sʰʌul listen lit capital off...
176,busan korean pronunciation pusa̠n officially k...
177,tokyo ˈtoʊkioʊ japanese 東京 tōkyō toːkʲoː liste...
178,kyoto ˈkjoʊtoʊ japanese 京都 kyōto kʲoꜜːto liste...


In [ ]:
# Save the cleaned text to a csv file
wk_df.to_csv("/content/drive/MyDrive/Priceline/JintongYu/wiki_cleaned.csv")